In [ ]:
from pynq import Overlay
import os
import sys

HOP_DIR=os.path.abspath("../")
sys.path.insert(0, HOP_DIR)
import pushpush

OVERLAY_DIR=f'{HOP_DIR}/overlays/add/'

# Preparation
#### Load the overlay onto the FPGA

In [ ]:
ol = Overlay(OVERLAY_DIR + "add.bit")
ol.ip_dict

#### Register the overlay with the HoP context

In [ ]:
context = pushpush.Context(ol)

# Use

#### Create some python functions

In [ ]:
def py_func1():
    print("Hello from func1")
    return 56

def py_func2():
    print("Hello from func2")    
    return 1000

def py_func3():
    print("Hello from func3")
    return 100

#### Register functions with HoP context

In [ ]:
a_py = context.register(py_func1, "() -> b32")
b_py = context.register(py_func2, "() -> b32")
c_py = context.register(py_func3, "() -> b32")

#### Register a variable

In [ ]:
x = context.register(52, 'b16')

In [ ]:
context.print_all_objects()

#### Get our hardware

In [ ]:
hw_add = context.functions['hardware']['add']

### Use Hardware as a regular function

In [ ]:
hw_add(b_py, 3)

In [ ]:
hw_add(a_py, b_py)

In [ ]:
hw_add(c_py, a_py)

# Debug

In [ ]:
import importlib as il
if 'context' in locals():
    del context
    il.reload(pushpush)
    pushpush.Context.reloadModules()
    context = pushpush.Context(ol)